In [1]:
import ee
import geemap
import pandas as pd
import json
import os

# GEE kimlik doğrulama ve proje kimliği ile başlatma
project_id = 'ee-turelhaticezehra'

# İlk seferde kimlik doğrulama işlemi yapılmalı
ee.Authenticate()

# GEE başlatma işlemi proje kimliği ile birlikte
ee.Initialize(project='ee-turelhaticezehra')

# Program başladığını belirt
print("Program başladı, işlemler devam ediyor...")

# İl ve ilçe sınırlarının yüklenmesi
with open('provinces.json', encoding='utf-8') as f:
    provinces_geojson = json.load(f)

with open('districts.json', encoding='utf-8') as f:
    districts_geojson = json.load(f)

# Bartın ilçelerini seç
bartin_districts = [
    feature for feature in districts_geojson['features'] 
    if feature['properties']['NAME_1'].lower() == 'bartın'
]

# Bartın ilçelerini kontrol etme
print(f"Bartın ilçeleri: {bartin_districts}")

# Tarih ve bulut filtresi parametreleri
date_ranges = [
    {'start': '2016-06-01', 'end': '2016-09-30', 'cloud': 10},
    {'start': '2017-06-01', 'end': '2017-09-30', 'cloud': 10},
    {'start': '2018-06-01', 'end': '2018-09-30', 'cloud': 10},
    {'start': '2019-06-01', 'end': '2019-09-30', 'cloud': 10},
    {'start': '2020-06-01', 'end': '2020-09-30', 'cloud': 10},
    {'start': '2021-06-01', 'end': '2021-09-30', 'cloud': 10},
    {'start': '2022-06-01', 'end': '2022-09-30', 'cloud': 10},
    {'start': '2023-06-01', 'end': '2023-09-30', 'cloud': 10},
    {'start': '2024-06-01', 'end': '2024-09-30', 'cloud': 10},
]

# NDVI eşik değerleri (Yeşil alan tanımı için NDVI > 0.2)
ndvi_threshold = 0.2
pixel_area = 30 * 30  # Sentinel-2 piksel boyutu: 30m x 30m (m²)

# Çıktı dosyası adı
output_csv = 'ndvi_results_bartin_il_ve_ilce.csv'

# Eğer dosya mevcutsa yükle, yoksa başlat
if os.path.exists(output_csv):
    ndvi_df = pd.read_csv(output_csv)
    print(f"Var olan {output_csv} dosyası yüklendi.")
else:
    ndvi_df = pd.DataFrame(columns=['year', 'province', 'district', 'mean_ndvi', 'total_green_area_ha'])
    print(f"Yeni {output_csv} dosyası oluşturuldu.")

# Bartın ilçeleri için NDVI hesaplama
print("Bartın ilçeleri için işlem başlıyor...")
for district in bartin_districts:
    district_name = district['properties']['NAME_2']  # İlçe adı
    district_geometry = ee.Feature(ee.Geometry.MultiPolygon(district['geometry']['coordinates']))
    
    for date_range in date_ranges:
        print(f"{district_name} ilçesi için {date_range['start']} ile {date_range['end']} tarih aralığı işleniyor...")
        
        # Sentinel-2 görüntü koleksiyonunu oluştur
        sentinel_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
            .filterBounds(district_geometry.geometry()) \
            .filterDate(date_range['start'], date_range['end']) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', date_range['cloud']))
        
        # Veri varsa, işleme devam et
        if sentinel_collection.size().getInfo() == 0:
            print(f"{district_name} ilçesi için {date_range['start']} ile {date_range['end']} tarih aralığında veri bulunamadı.")
            continue  # Verisi olmayan ilçeyi atla
        
        # NDVI hesaplama
        sentinel_mosaic = sentinel_collection.median().clip(district_geometry.geometry())
        ndvi = sentinel_mosaic.normalizedDifference(['B8', 'B4']).rename('NDVI')
        
        # Yeşil alan hesaplaması
        green_area = ndvi.gt(ndvi_threshold).multiply(pixel_area).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=district_geometry.geometry(),
            scale=30,
            maxPixels=1e13
        ).get('NDVI').getInfo()
        
        if green_area is None:
            green_area = 0
        
        # Ortalama NDVI hesaplama
        mean_ndvi = ndvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=district_geometry.geometry(),
            scale=30,
            maxPixels=1e13
        ).get('NDVI').getInfo()
        
        if mean_ndvi is None:
            mean_ndvi = 0  # Eğer yeşil alan yoksa sıfırla
        
        # Yeni verileri ekle
        new_data = {
            'year': date_range['start'][:4],
            'province': "Bartın",
            'district': district_name,
            'mean_ndvi': mean_ndvi,
            'total_green_area_ha': green_area / 1e4  # m² -> hektar
        }
        ndvi_df = pd.concat([ndvi_df, pd.DataFrame([new_data])], ignore_index=True)
        print(f"{district_name} ilçesi için {date_range['start']} ile {date_range['end']} tarih aralığı sonuçları kaydedildi.")

# İlçeler için NDVI analizleri tamamlandı
print("İlçeler için NDVI analizleri tamamlandı. Veriler kaydediliyor...")

# İlçelerden il geneli değerleri hesaplama
province_results = []

for date_range in date_ranges:
    year = date_range['start'][:4]
    province_data = ndvi_df[(ndvi_df['year'] == year) & (ndvi_df['province'] == 'Bartın')]
    
    total_green_area = province_data['total_green_area_ha'].sum()
    # Ağırlıklı ortalama NDVI hesaplama
    if total_green_area > 0:
        weighted_mean_ndvi = (province_data['mean_ndvi'] * province_data['total_green_area_ha']).sum() / total_green_area
    else:
        weighted_mean_ndvi = 0
    
    province_results.append({
        'year': year,
        'province': "Bartın",
        'district': "Toplam",  # İl düzeyi için boş
        'mean_ndvi': weighted_mean_ndvi,
        'total_green_area_ha': total_green_area
    })

# İl geneli sonuçları DataFrame'e ekleme
ndvi_df = pd.concat([ndvi_df, pd.DataFrame(province_results)], ignore_index=True)

# Sonuçları kaydetme
ndvi_df.to_csv(output_csv, index=False)
print("Bartın ili için toplam NDVI ve yeşil alan hesaplamaları tamamlandı ve sonuçlar kaydedildi.")


Program başladı, işlemler devam ediyor...
Bartın ilçeleri: [{'type': 'Feature', 'properties': {'GID_2': 'TUR.13.1_1', 'GID_0': 'TUR', 'COUNTRY': 'Turkey', 'GID_1': 'TUR.13_1', 'NAME_1': 'Bartın', 'NL_NAME_1': 'NA', 'NAME_2': 'Amasra', 'VARNAME_2': 'NA', 'NL_NAME_2': 'NA', 'TYPE_2': 'District', 'ENGTYPE_2': 'District', 'CC_2': 'NA', 'HASC_2': 'TR.BR.AM'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[32.5461, 41.6993], [32.5171, 41.6921], [32.5084, 41.6883], [32.5017, 41.6829], [32.4922, 41.6691], [32.4807, 41.658], [32.469, 41.6509], [32.4543, 41.6474], [32.3743, 41.648], [32.3381, 41.6577], [32.2763, 41.6606], [32.2296, 41.674], [32.2282, 41.6776], [32.2235, 41.6799], [32.226, 41.6813], [32.2238, 41.6835], [32.2271, 41.684], [32.226, 41.6854], [32.2243, 41.6849], [32.2263, 41.6865], [32.2318, 41.6857], [32.2368, 41.6829], [32.236, 41.6851], [32.2321, 41.6857], [32.2324, 41.6888], [32.2504, 41.6954], [32.2582, 41.7038], [32.2637, 41.7054], [32.2651, 41.709], [32.2693, 41.7096